In [1]:
# With varying Cl and Cd for NREL 1.5MW wind turbine blade:
# here Vo, chord, beta, rad_loc, R, wr, pitch angle has to be changed accordingly

import numpy as np, pandas as pd
from math import *
S818=pd.read_csv('S818.csv')      # Importing Cl and Cd data for S818
S825=pd.read_csv('S825.csv')      # Importing Cl and Cd data for S825
S826=pd.read_csv('S826.csv')      # Importing Cl and Cd data for S826
df=pd.DataFrame(columns=['rad_loc','Airfoil','chord','alpha','beta','n','Cl','Cd','a',
                         'ai','F','phi','Vrel','L','D','Pn','Pt'])

In [2]:
S818

,Alpha,Cl,Cd,Cm
0,-14.50,-0.7526,0.08924,-0.0372
1,-14.25,-0.7493,0.08733,-0.0370
2,-14.00,-0.7445,0.08532,-0.0371
3,-13.75,-0.7194,0.08180,-0.0432
4,-13.50,-0.7022,0.07837,-0.0480
...,...,...,...,...
118,15.75,1.5095,0.06527,-0.0902
119,16.00,1.5035,0.06943,-0.0907
120,16.25,1.5033,0.07297,-0.0913
121,16.50,1.5017,0.07671,-0.0920


In [3]:
mf=pd.read_csv('No_Interpolated_input_data.csv')
mf

,SPAN,TWIST,CHORD,AIRFOIL
0,3.09375,42.00,2.5328,S818
1,5.15625,32.00,2.8157,S818
2,7.21875,23.00,3.0740,S818
3,9.28125,15.00,3.2101,S818
4,11.34375,11.50,3.1115,S818
5,13.40625,8.20,2.9651,S818
6,15.46875,7.00,2.8182,S818
7,17.53125,6.00,2.6726,S818
8,19.59375,5.00,2.5270,S825
9,21.65625,4.00,2.3805,S825


In [4]:
for p in range(19):
    #print(df.loc[0,:])
    rad_loc=mf.loc[p,:]['SPAN']
    beta=mf.loc[p,:]['TWIST']
    chord=mf.loc[p,:]['CHORD']
    KF=[S818,S825,S826]
    kf=['S818','S825','S826']
    if p==0:
        AF=KF[0]
        af=kf[0]
        #print(p)
        #print(AF)
    elif p==8:
        AF=KF[1]
        af=kf[1]
        #print(p)
        #print(AF)
    elif p==16:
        AF=KF[2]
        af=kf[2]
        #print(p)
        #print(AF)
    #AF=mf.loc[p,:]['AIRFOIL']
    #print(rad_loc)
    #print(beta)
    #print(chord)
    a=0
    ai=0            
    Vo=12          # Free stream velocity
    B=3            # Number of Blades
    #chord=2.9651      # Chord (Changes at each section)
    #beta=8.2         # Twist angle (Changes at each section)
    #rad_loc=13.40625   # span wise location from the root (Changes at each section)
    R=41.25          # Total length of the blade.
    wr=1.958*rad_loc 
    sig=(chord*B)/(2*pi*rad_loc) # Solidity factor
    i=1
    n=1
    while i==1:
        phi=(degrees(atan((((1-a)*Vo)/((1+ai)*wr)))))      # Flow angle

        # Prandlt's tip loss factor
        f=(B*(R-rad_loc))/(2*rad_loc*np.sin(np.deg2rad(phi)))
        F=(2/pi)*acos(exp(-f))

        pitch_angle=0                            # pitch angle is angle b/w tip chord and rotar plane         .
        local_pitch = pitch_angle+beta           # beta is measured relative to tip chord
        alpha=phi-local_pitch;                   # Angle of attack 

        if round(alpha,2) in list(AF['Alpha']):
            Cl=np.array(AF[AF['Alpha']==round(alpha,2)]['Cl'])[0]
            Cd=np.array(AF[AF['Alpha']==round(alpha,2)]['Cd'])[0]
            Cm=np.array(AF[AF['Alpha']==round(alpha,2)]['Cm'])[0]
        else:
            a_list=list(AF['Alpha'])
            given_value=alpha
            absolute_difference_function = lambda list_value : abs(list_value - given_value)
            closest_value = min(a_list, key=absolute_difference_function)
            alpha=closest_value
            Cl=np.array(AF[AF['Alpha']==round(alpha,2)]['Cl'])[0]
            Cd=np.array(AF[AF['Alpha']==round(alpha,2)]['Cd'])[0]
            Cm=np.array(AF[AF['Alpha']==round(alpha,2)]['Cm'])[0]

        Cn=Cl*np.cos(np.deg2rad(phi))+Cd*np.sin(np.deg2rad(phi))
        Ct=Cl*np.sin(np.deg2rad(phi))-Cd*np.cos(np.deg2rad(phi))
        ac=0.2
        if a<ac:
            new_a=1/(((4*F*np.sin(np.deg2rad(phi))*np.sin(np.deg2rad(phi)))/(sig*Cn))+1);
            new_ai=1/(((4*F*np.sin(np.deg2rad(phi))*np.cos(np.deg2rad(phi)))/(sig*Ct))-1);
        else:
            # Glauert correction for high value of a
            K=(4*F*np.sin(np.deg2rad(phi))*np.sin(np.deg2rad(phi)))/(sig*Cn)
            new_a=0.5*(2+(K*(1-(2*ac)))-sqrt((((K*(1-(2*ac)))+2)**2)+(4*((K*ac*ac)-1))))
            new_ai=1/(((4*F*np.sin(np.deg2rad(phi))*np.cos(np.deg2rad(phi)))/(sig*Ct))-1)
        
        cop=0.25-(Cm/((Cl*np.cos(np.deg2rad(alpha)))+(Cl*np.cos(np.deg2rad(alpha)))))
        #cop=cop*chord

        if abs(new_a-a) < 0.00001 and abs(new_ai-ai) < 0.00001:
            i=0
            break
        a=new_a
        ai=new_ai
        n=n+1

        if n==1000:
            break

    Vrel=sqrt((((1-new_a)*Vo)**2)+(((1-new_ai)*wr)**2))
    L=0.5*1.225*chord*Cl*Vrel*Vrel
    D=0.5*1.225*chord*Cd*Vrel**2
    Pn=(L*np.cos(np.deg2rad(phi)))+(D*np.sin(np.deg2rad(phi)))
    Pt=(L*np.sin(np.deg2rad(phi)))-(D*np.cos(np.deg2rad(phi)))
    
    #print(af)
    df = df.append({'rad_loc':rad_loc,'Airfoil':af,'chord':chord,'alpha':alpha,'beta':beta,'n' : n, 'Cl' : Cl, 'Cd' : Cd, 
                    'a':a,'ai':ai,'F':F,'phi':phi,'Vrel':Vrel,'L':L,'D':D,'Pn':Pn,'Pt':Pt}, 
                    ignore_index = True)
df

,rad_loc,Airfoil,chord,alpha,beta,n,Cl,Cd,a,ai,F,phi,Vrel,L,D,Pn,Pt
0,3.09375,S818,2.5328,10.75,42.00,7,1.4973,0.01846,0.123932,0.315682,1.000000,52.834018,11.300568,296.630902,3.657120,182.116782,234.172466
1,5.15625,S818,2.8157,9.75,32.00,5,1.4742,0.01409,0.140406,0.145946,1.000000,41.719803,13.444333,459.544310,4.392199,345.930605,302.543023
2,7.21875,S818,3.0740,9.50,23.00,8,1.4526,0.01380,0.178127,0.094405,0.999999,32.520688,16.158881,714.132081,6.784402,605.801629,378.199740
3,9.28125,S818,3.2101,10.25,15.00,11,1.4930,0.01590,0.233897,0.071384,0.999996,25.275572,19.217060,1084.072617,11.545047,985.217986,452.429272
4,11.34375,S818,3.1115,9.50,11.50,11,1.4526,0.01380,0.256332,0.052236,0.999990,20.898690,22.864256,1447.225083,13.748937,1356.920426,503.404838
5,13.40625,S818,2.9651,9.50,8.20,1000,1.4526,0.01380,0.279822,0.040615,0.999979,17.604358,26.624910,1870.112651,17.766456,1787.904238,548.666939
6,15.46875,S818,2.8182,8.50,7.00,12,1.3580,0.01300,0.279759,0.030488,0.999946,15.478267,30.609906,2196.350550,21.025447,2122.303878,565.883432
7,17.53125,S818,2.6726,7.75,6.00,9,1.2823,0.01259,0.277628,0.023626,0.999867,13.858351,34.618058,2515.562441,24.698535,2448.252408,578.553852
8,19.59375,S825,2.5270,7.00,5.00,14,1.3544,0.01226,0.312381,0.020822,0.999795,11.897769,38.461281,3101.028611,28.070445,3040.196427,611.859525
9,21.65625,S825,2.3805,6.75,4.00,14,1.3388,0.01145,0.319211,0.017375,0.999567,10.723255,42.459500,3519.165795,30.097437,3463.311955,625.223203
